<a href="https://colab.research.google.com/github/weldlabucsb/mcmc/blob/master/mcmc_nuts_clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
#Goal of this notebook is to create some simple code that can take a 1d array which we will suppose we have, i.e. it will be one of the inputs
#and we will take it and compare it to simulated data.

## Imports:

import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
from scipy.integrate import odeint
from scipy.interpolate import interp1d
from scipy.integrate import solve_ivp
from multiprocessing import Process, Queue
import time
import matplotlib.pyplot as plt
from IPython.core.pylabtools import figsize

class _TFColor(object):
    """Enum of colors used in TF docs."""
    red = '#F15854'
    blue = '#5DA5DA'
    orange = '#FAA43A'
    green = '#60BD68'
    pink = '#F17CB0'
    brown = '#B2912F'
    purple = '#B276B2'
    yellow = '#DECF3F'
    gray = '#4D4D4D'
    def __getitem__(self, i):
        return [
            self.red,
            self.orange,
            self.green,
            self.blue,
            self.pink,
            self.brown,
            self.purple,
            self.yellow,
            self.gray,
        ][i % 9]
TFColor = _TFColor()

tfd = tfp.distributions

# Global Constants

In [ ]:
# number of "pixels" on the direction
N = 150

# Data Import

We will take some data and turn it a 1d array with the data along a given direction.

The idea is that the data is a 2d matrix, we will interpolate it to get a function OD(x, y) that will provide the optical density at pixels x,y. We want to be able to slice it, so that we give a direction like \[0,1] or \[1,1] and with N bins. Each bin would represent a pixel. 

We have used the DataManager made by Peter to get a CSV file that I will use for this run.

Load the optical density (could add some kind of interworking with matlab maybe).

All of the od < 0 should be equal to 0.

In [ ]:
#load csv file:
od = np.loadtxt(r"/content/drive/My Drive/Colab Notebooks/Weld Lab/data/optical density run 18.csv", delimiter=",", encoding='utf-8-sig').T
od[od < 0] = 0 #isn't this so easy with numpy?? How do you even do the same on matlab? Is it as easy, maybe?
od.shape

(41, 390)

In [ ]:
#import scipy:
from scipy.interpolate import RectBivariateSpline

In [ ]:
#@title Interpolation of OD
X = range(od.shape[0])
Y = range(od.shape[1])
optical_density = RectBivariateSpline(X, Y, od)

## Helper function to cut through the data

In [ ]:
#this function will provide an array of points, so X, and Y where X[i], Y[i]
#are places where to evaluate this the function along a given direction.
def points(start, end, Nx, Ny, N):
  x1 = start[0]
  y1 = start[1]
  x2, y2 = end
  line = lambda x: (y2-y1)/(x2-x1)*(x-x1)+y1
  
  
  xinit = (x2*y1-x1*y2)/(y1-y2)
  xfinal = (Ny*(x1-x2)-x1*y2+x2*y1)/(y1-y2)
  print(f"xinit is {xinit:.2f}, xfinal is {xfinal:.2f}.")
  if xinit<0:
    xinit = 0
  elif xinit > Nx:
    xinit = Nx
  if xfinal > Nx:
    xfinal = Nx
  elif xfinal < 0:
    xfinal = 0
  
  X = np.linspace(xinit, xfinal, N)
  Y = line(X)
  return (X, Y)

## Helper function to cut through the data

In [ ]:
X, Y = points((15,196),(25,189), od.shape[0], od.shape[1], int(N))
plt.figure(figsize=(5,5))
plt.plot(X, Y)
plt.xlim(0, od.shape[0])
plt.ylim(0, od.shape[1])
plt.figure(figsize=(5,5))
Slice = optical_density(X, Y, grid=False)
plt.plot(Slice)

In [ ]:
#normalize the slice: (add option to normalise with true dx later)
from scipy.integrate import trapz
Slice = Slice/trapz(Slice)
#make sure there's no negative numbers:
Slice[Slice < 0] = 0
plt.plot(Slice)
trapz(Slice)

# Markov Chain Monte Carlo Set Up



## Initial Step Set Up

In [ ]:
X = tf.cast(tf.constant(range(N)), dtype=tf.float32)
print(X)
sigma = 8.
def gaussian_rates(sigma, mu, w):
  # Xmod = tf.transpose(X*tf.ones(( mu.shape[0], X.shape[0])))
  # print(Xmod)
  # Mumod = mu*tf.ones((X.shape[0], mu.shape[0]))
  # print(Mumod)
  # I0 =  Xmod - Mumod
  # print(I0)
  # I1 = -1.*(tf.pow(I0, 2.)/sigma)
  return w*tf.exp(  -1.*tf.pow(tf.transpose(X*tf.ones(( mu.shape[0], X.shape[0])))-
                    mu*tf.ones((X.shape[0], mu.shape[0])), 2.)/sigma )

def sim_od_dist(sigma, mu, w):
  rate = w*tf.exp(-(tf.pow(X-mu, 2.)/sigma))
  rate = rate.numpy()
  rate[rate < 1e-8] = 1e-10
  #let's try 3 poisson distributions centered at different places.

  # print("rate ", rate)
  return tfd.Independent(tfd.Poisson(rate=tf.convert_to_tensor(rate, dtype=tf.float32)), reinterpreted_batch_ndims=1)
  # return tfd.Independent(tfd.Normal(loc=w*tf.exp(-(tf.pow(X-mu, 2.)/sigma)), scale=2.), reinterpreted_batch_ndims=1)
def sim_od_multiple_dist(sigma, mu, w):
  # rates = gaussian_rates(sigma, mu, w)
  return tfd.Independent(tfd.Normal(loc=tf.reduce_sum(gaussian_rates(sigma,
                                                                     mu, w),
                                                      axis=1),
                                    scale=tf.ones((int(N),))*0.2), reinterpreted_batch_ndims=1)



dists = [
         #w
         tfd.Uniform(low=0.0, high=10000.0),
         #mu
         tfd.Normal(loc=50., scale=8.),
         #sigma
         tfd.Normal(loc=20., scale=10.),
         #total atoms
         tfd.Normal(loc=10000., scale=50.),
         lambda atoms, sigma, mu, w: sim_od_dist(sigma, mu, w)
]
def generate_multiple_dists(n: int):
  return [
         #w
         tfd.Independent(tfd.Uniform(low=[0.0]*n, high=[10000.]*n),reinterpreted_batch_ndims=1),
         #mu
         tfd.Independent(tfd.Normal(loc=[80.]*n, scale=[50.]*n), reinterpreted_batch_ndims=1),
         #sigma
         tfd.Independent(tfd.Normal(loc=[50.]*n, scale=[5.]*n), reinterpreted_batch_ndims=1),
         #total atoms
         tfd.Normal(loc=1000., scale=80.),
         lambda atoms, sigma, mu, w: sim_od_multiple_dist(sigma, mu, w)
        ]


od_dist = tfd.JointDistributionSequential(dists)
od_multiple_dist = tfd.JointDistributionSequential(generate_multiple_dists(3))
print(od_multiple_dist.resolve_graph())
s = od_multiple_dist.sample(1)
print("sample ", s)
od_multiple_dist.log_prob_parts(s)


In [ ]:
def target_log_prob_tr(*init_state):
  try:
    #print("init st = ", init_state, "\n len init", len(init_state))
    atoms = init_state[3]
    inp = list(init_state)+[Slice*atoms]
    return od_dist.log_prob(inp)
    #print("inp ", inp, "\n", len(inp))
  except Exception as e:
    # print(e)
    atoms = init_state[0][3]
    # print(atoms)
    inp = list(init_state[0])+[Slice*atoms]
    # print(inp)
    try:
      return od_dist.log_prob(inp)
    except Exception as e:
      print(e)
      print(init_state)
      exit()

def target_log_prob_multiple_normals(*init_state):
  try:
    # print("init st = ", init_state, "\n len init", len(init_state))
    atoms = init_state[3]
    inp = list(init_state)+[Slice*atoms]
    return od_multiple_dist.log_prob(inp)
    #print("inp ", inp, "\n", len(inp))
  except Exception as e:
    # print(e)
    atoms = init_state[0][3]
    # print(atoms)
    inp = list(init_state[0])+[Slice*atoms]
    # print(inp)
    try:
      return od_multiple_dist.log_prob(inp)
    except Exception as e:
      print(e)
      print(init_state)
      exit()

In [ ]:
#creation of the distribution used:
# dist = closed_joint_dist_gen(int(N)) #N is the number of pixels in the slice, set globally
# dist = 
#how many chains should we use?
chains = 1
#sample it:
# start = dist.sample()
step_size = 0.1

In [ ]:
#@title Bijectors
# Since HMC operates over unconstrained space, we need to transform the
# samples so they live in real-space.
# unconstraining_bijectors = [
#     tfp.bijectors.Exp(),       # Maps a positive real to R. for x_tail
#     tfp.bijectors.Exp(), # for lambda center
#     tfp.bijectors.Exp(), # for lambda left
#     tfp.bijectors.Exp(), # for lambda right
#     tfp.bijectors.Exp(), # for sigma center
#     tfp.bijectors.Exp(), # for sigma left
#     tfp.bijectors.Exp(), # for sigma right
#     tfp.bijectors.Exp(), # for sigmai
#     tfp.bijectors.Exp(), # for amplitude
#     tfp.bijectors.Sigmoid(), #for mix center
#     tfp.bijectors.Sigmoid() #for mix left
#     # tfp.bijectors.Sigmoid(),       # Maps [0,1] to R. for gamma
#     # tfp.bijectors.Sigmoid(),   # Maps [0,1] to R. for delta
#     # tfp.bijectors.Exp(),        # Maps a positive real to R. for I0
#     # tfp.bijectors.Exp(), # Maps a positive real to R. for R0
#     # tfp.bijectors.Sigmoid() # Maps [0,1] to R. for care
# ]
unconstraining_bijectors = [
    tfp.bijectors.Exp(),       # Maps a positive real to R. for x_tail
    tfp.bijectors.Exp(), # for mu
    tfp.bijectors.Exp(), # for sigma
    tfp.bijectors.Exp() # for atom number
    # tfp.bijectors.Exp(), # for lambda right
    # tfp.bijectors.Exp(), # for sigma center
    # tfp.bijectors.Exp(), # for sigma left
    # tfp.bijectors.Exp(), # for sigma right
    # tfp.bijectors.Exp(), # for sigmai
    # tfp.bijectors.Exp(), # for amplitude
    # tfp.bijectors.Sigmoid(), #for mix center
    # tfp.bijectors.Sigmoid() #for mix left
    # tfp.bijectors.Sigmoid(),       # Maps [0,1] to R. for gamma
    # tfp.bijectors.Sigmoid(),   # Maps [0,1] to R. for delta
    # tfp.bijectors.Exp(),        # Maps a positive real to R. for I0
    # tfp.bijectors.Exp(), # Maps a positive real to R. for R0
    # tfp.bijectors.Sigmoid() # Maps [0,1] to R. for care
]

In [ ]:
#@title Starting MCMC Code
from google.colab import output
chains = 1
local_runtime = False #@param {type:"boolean"}
# wrap the mcmc sampling call in a @tf.function to speed it up
#@tf.function(autograph=False)
calls = 0
def graph_sample_chain(*args, **kwargs):
  print("current = ",kwargs["current_state"])
  start_time = time.time()
  out = tfp.mcmc.sample_chain(*args, **kwargs)
  print(f"It took: {(time.time()-start_time)/60} min")
  return out
def trace_fn(_, pkr):
    return (
        #pkr.inner_results.inner_results.target_log_prob,
        pkr.inner_results.inner_results.leapfrogs_taken,
        pkr.inner_results.inner_results.has_divergence,
        pkr.inner_results.inner_results.energy,
        pkr.inner_results.inner_results.log_accept_ratio
           )
# r0, gamma, delta, I0, rho, _ = mdl_ols_batch.sample(nchain)
# init_state = [b0, b1]
# # step_size = [tf.cast(i, dtype=dtype) for i in [.1, .1]]
# target_log_prob_fn = lambda *init_state: mdl_ols_batch.log_prob(
#     list(init_state) + [Y_np])
num_burnin_steps = 10000
num_results = 5000



print(step_size)
kernel=tfp.mcmc.TransformedTransitionKernel(
        inner_kernel=tfp.mcmc.HamiltonianMonteCarlo(
            target_log_prob_fn=target_log_prob_multiple_normals,
            num_leapfrog_steps=2,
            step_size=step_size,
            state_gradients_are_stopped=False),
        bijector=unconstraining_bijectors)

kernel = tfp.mcmc.SimpleStepSizeAdaptation(
    inner_kernel=kernel, num_adaptation_steps=int(num_burnin_steps * 0.8))

#kernel.bootstrap_results(init_state)

# # Sample from the chain.
# [
#     r0_samples,
#     gamma_samples,
#     delta_samples,
#     I0_samples,
#     posterior_care,
# ],
kernel_results = graph_sample_chain(
    num_results=num_results,
    num_burnin_steps=num_burnin_steps,
    current_state=init, #put init in here if you want to start from init
    #previous_kernel_results=kernel.bootstrap_results(initial_chain_state),
    kernel = kernel, parallel_iterations=chains)
#save the kernel_results to drive:
[w_samples, mu_samples, sigma_samples, atom_samples], extra = kernel_results
# [xtail_samples, lambdac_samples, lambdal_samples, lambdar_samples, sigmac_samples, sigmal_samples, sigmar_samples, sigmai_samples, amplitude_samples, mixc_samples, mixl_samples], extra = kernel_results
#save data as plain npz files:
base = r"Desktop/covid19/" if local_runtime else r"/content/drive/My Drive/Colab Notebooks/Weld Lab/data/"
output_path = base + f"out/output_run18_{time.strftime('%Y-%m-%d-%H:%M %Z', time.localtime())}.npz"
np.savez(output_path, w_samples, mu_samples, sigma_samples, atom_samples)# xtail_samples, lambdac_samples, lambdal_samples, lambdar_samples, sigmac_samples, sigmal_samples, sigmar_samples, sigmai_samples, amplitude_samples, mixc_samples, mixl_samples)

# notify.send("Finished!")
#np.savez("/content/drive/My Drive/Colab Notebooks/out/output_acceptance.npz", tf.cast(extra.inner_results.inner_results.is_accepted,dtype=tf.float32)).numpy(), extra.inner_results.inner_results.accepted_results.step_size[-100:])
# tau_samples = tf.floor(posterior_care * tf.cast(tf.size(cases),dtype=tf.float32))
output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')

In [ ]:
#@title Calculate Acceptance Rate
print("acceptance rate: {}".format(
    tf.reduce_mean(tf.cast(extra.inner_results.inner_results.is_accepted,dtype=tf.float32))))
print("final step size: {}".format(
    tf.reduce_mean(extra.inner_results.inner_results.accepted_results.step_size[-100:])))
# extra
# step_size = [o[-1] for o in extra.inner_results.inner_results.accepted_results.step_size]
step_size =  tf.reduce_mean(extra.inner_results.inner_results.accepted_results.step_size[-100:])
tf.reduce_mean(extra.inner_results.inner_results.accepted_results.target_log_prob, axis=0)
# step_size = 0.1